In [1]:
import torch

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [2]:
# IMPORTS

# Files
import data_preprocess

# Torch Features
from torch import nn
from torch import optim
import torch.nn.functional as F

# Calc Utilities
import numpy as np
from collections import OrderedDict

# Plot Utilities
import matplotlib.pyplot as plt
%matplotlib inline

# Miscellaneous
import time


In [ ]:
def train_model(model, n_epochs=80, checkpoint_path = False):
    
    # Start epoch variable
    start_epoch = 0
    
    # Track change in validation loss
    valid_loss_min = np.Inf 
    
    # Define the Loss
    criterion = nn.NLLLoss()

    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adadelta(model.fc.parameters())
    
    #Check if there is an old run
    if checkpoint_path:
      cfile = torch.load(checkpoint_path)
      start_epoch = cfile['epochs']
      optimizer.load_state_dict(cfile['optim'])

    
    for epoch in range(1, n_epochs + 1):
        start_time = time.time()
      
        # Move model to GPU if CUDA is available
        if train_on_gpu:
            model.cuda()

        # keep track of training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        ###################
        # train the model #
        ###################
        model.train()
        for data, target in train_loader:
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            # flatten the data
            data = data.view(-1, 9, 1, 128)
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss += loss.item()*data.size(0)
            
            # data to train acc.
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum()
        
        # calculate train acc.
        acc_train = float(correct) * 100.0 / (batch_size * n_batch)
        
        ######################    
        # validate the model #
        ######################
        correct = 0
        total = 0
        model.eval()
        for data, target in valid_loader:
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            # flatten the data
            data = data.view(-1, 9, 1, 128)
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss += loss.item()*data.size(0)
            
            #calculate accuracy
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum()

        # calculate average losses
        train_loss = train_loss/len(train_loader.dataset)
        valid_loss = valid_loss/len(valid_loader.dataset)
        acc_valid = float(correct) * 100.0 / (batch_size * n_batch)
        duration = (time.time() - start_time)/60       


        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tTrain Acc.: {:.2f} \tValidation Acc.: {:.2f} \tDuration: {:.2f} min.'.format(
            start_epoch + epoch, train_loss, valid_loss,acc_train,acc_valid,duration))

        # save model if validation loss has decreased



In [4]:
if __name__ == '__main__':
    # how many samples per batch to load
    batch_size = 20
    # define the number of batches
    n_batch = len(train_loader.dataset) // batch_size
    # number of epochs
    n_epochs = 80 
    
    # Load data
    train_loader, valid_loader, test_loader = data_preprocess.load(batch_size=batch_size)
    # Define the Model
    
    # Train the Model
    train_model(model, n_epochs, checkpoint_path = False)

(7352, 1152)
(7352, 128, 9)
(2947, 1152)
(2947, 128, 9)
